<a href="https://colab.research.google.com/github/Silvarawenne/Projetos-Data-Science/blob/main/Projeto_Previsao_Precos_Imoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 1: Previsão de Preços de Imóveis
Este projeto utiliza o dataset Ames Housing do Kaggle para construir um modelo de machine learning que prevê o preço de venda de casas.

In [ ]:
from google.colab import files
files.upload()

In [ ]:
# Cria a pasta .kaggle, move o arquivo e define as permissões
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

In [ ]:
# Desconpactando arquivo .zip

!unzip house-prices-advanced-regression-techniques.zip

In [ ]:
#Importando panda e carregando arquivos de treino para um DataFrame

import pandas as pd

df = pd.read_csv('train.csv')

print("Dados de treino carregados!As 5 primeiras linhas são:")
df.head()

In [ ]:
# Mostrando um resumo técnico da lista

print("/nResumo técnico do DataFrame:")
df.info()

In [ ]:
# Mostra as estatiscas descritivas
print("Estatisticas do preço de venda (salePrice):")
df['SalePrice'].describe()

In [ ]:
# Criando um gráfico para ver a distribuição de coluna "salePrices"
import seaborn as sns
import matplotlib.pyplot as plt


plt.figure(figsize=(10, 6))
sns.histplot(df['SalePrice'], kde=True, bins=50)
plt.title('Distribuição dos preços de venda (SalePrice)', fontsize=16)
plt.xlabel('Preço de venda(USD)')
plt.ylabel('Frequência (Número de casas)')
plt.show()

In [ ]:
# Aplicando e transformando log na coluna salesprice e após isso criando uma figura com dois gráficos lado a lado

import numpy as np
df['SalePrice_log'] = np.log1p(df['SalePrice'])

fig, axes = plt.subplots(1,2,figsize=(15, 6))

sns.histplot(df['SalePrice'], kde= True, ax=axes[0])
axes[0].set_title('Distribuição Original de SalePrice')
axes[0].set_xlabel('Preço das vendas USD')

sns.histplot(df['SalePrice_log'], kde=True, ax=axes[1], color='green')
axes[1].set_title('Distribuição de SalePrice após Transformação Logaritmica')
axes[1].set_xlabel('Log dp preço de venda')
plt.show()


## Análise Exploratória: Encontrando as Melhores Pistas
A seguir, investigamos a correlação entre as características da casa e o preço de venda.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

correlacoes = df.select_dtypes(include=['number']).corr()

print("Top 10 features mais correlacionados com o preço de venda:")
print(correlacoes['SalePrice'].sort_values(ascending=False).head(11))

In [ ]:
top_10_features = correlacoes['SalePrice'].sort_values(ascending=False).head(11).index
top_10_corr_matrix = df[top_10_features].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(top_10_corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
plt.title('Mapa de calor da 10 features mais correlacionadas com SalePrice', fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(x='OverallQual', y='SalePrice', data=df, palette='coolwarm')
plt.title('Preço de venda VS. qualidade Geral da Casa', fontsize=16)
plt.xlabel('Qualidade Geral da casa (1=Muito Ruim. 10=Excelente)')
plt.ylabel('Preço de venda (USD)')
plt.show()


In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='GrLivArea', y='SalePrice', data=df)
plt.title('Preço de venda VS. Área de Estar', fontsize=16)
plt.xlabel('Área de estar(Pés quadrados)')
plt.ylabel('Preço de venda (USD)')
plt.show()

In [ ]:
#-------------PREPARANDO O MODELO---------------
features_selecionadas= [
    'OverallQual',
    'GrLivArea',
    'GarageCars',
    'GarageArea',
    'TotalBsmtSF',
    'FullBath',
    'YearBuilt'
]

df_modelo = df[features_selecionadas + ['SalePrice']]

print("Verificando dados faltantes nas features selecionadas:")
df_modelo.isnull().sum()

In [ ]:
from re import X
from sklearn.model_selection import train_test_split

# separando as features do nosso target
X = df_modelo[features_selecionadas]
y = df_modelo['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Dados divididos com sucesso!")
print(f"Temos {len(X_train)} casas para treinar o modelo")
print(f"Temos {len(X_test)} casas para testar o modelo")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

#criando o modelo de floresta aleatória
modelo = RandomForestRegressor(n_estimators=100, random_state=42)

#Treinando p modelo com os dados de treino
print("Treinando o modelo...(Isso pode levar alguns segundos)")
modelo.fit(X_train, y_train)
print("Modelo treinado com sucesso!")

print("Fazendo previsões nos dados de teste...")
previsoes = modelo.predict(X_test)

mae = mean_absolute_error(y_test, previsoes)

print("---Resultado Final ---")
print(f"O erro absoluto (mae) do nosso modelo é: ${mae:,.2f}")
print("Isso significa que, na média, as previsões dos preços do nosso modelo erram este valor (para mais ou para menos) em relação ao preço real de casa.")